In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
#df.index = pd.to_datetime(df.index)

In [ ]:
df=pd.read_csv("/kaggle/input/weather-forecasting/weather.csv")
df.columns = df.columns.str.strip() # Remove leading/trailing spaces from column names

In [ ]:
df.head()

In [ ]:
df.drop(columns=['_wgustm','_windchillm','_precipm','_heatindexm','_conds','_dewptm','_fog','_hail','_pressurem','_rain','_snow','_tempm','_thunder','_tornado','_vism','_wdird','_wdire','_wspdm'] ,inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df = df.set_index('datetime_utc')#تتخليه يبدء اول عمود
df.index = pd.to_datetime(df.index)#غشان تخلي السنين تتعرض بشكل احسن في محور اكس 
#convert the index labels of a pandas DataFrame into datetime objects

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.plot(style='.',
        figsize=(15, 5),
        color=color_pal[0],
        title='PJME Energy Use in MW')
plt.show()

In [ ]:
train = df.loc[df.index < '01-01-2015']
test = df.loc[df.index >= '01-01-2015']

fig, ax = plt.subplots(figsize=(15, 5))
train.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
test.plot(ax=ax, label='Test Set')
ax.axvline('01-01-2015', color='black', ls='--')#الفاصل بين الاتنين 
ax.legend(['Training Set', 'Test Set'])
plt.show()

In [ ]:
df.loc[(df.index > '01-01-2010') & (df.index < '01-08-2010')] \
    .plot(figsize=(15, 5), title='Week Of Data')
plt.show()

In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df, x='hour', y='_hum')
ax.set_title('MW by Hour')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df, x='month', y='_hum', palette='Blues')
ax.set_title('MW by Month')
plt.show()

In [ ]:
train = create_features(train)
test = create_features(test)

FEATURES = ['hour','dayofweek','quarter','month','year','dayofyear']
TARGET = '_hum'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

In [ ]:
'''xgb.XGBRegressor-> is a powerful tool for solving regression problems by employing the XGBoost algorithm.
It combines the strengths of gradient boosting, feature engineering, and regularization
to create accurate predictive models for numerical output variables.'''
reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.01)#فنكشن تقييم 
reg.fit(X_train, y_train,#fit->used to train a machine learning model on a dataset
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)#كدا معناها بتطلع لكل واحد وتجيب العنصر اللي بعدو ب100

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

In [ ]:
'''In essence, the "Forecast on Test" step is a crucial part of the predictive modeling process.
It helps you assess how well your trained model generalizes to new, unseen data.
The goal is to build a model that can make accurate and reliable predictions on real-world data beyond the training dataset.'''
test['prediction'] = reg.predict(X_test)
df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
ax = df[['_hum']].plot(figsize=(15, 5))
df['prediction'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Dat and Prediction')
plt.show()

In [ ]:
ax = df.loc[(df.index > '04-01-2015') & (df.index < '04-08-2015')]['_hum'] \
    .plot(figsize=(15, 5), title='Week Of Data')
df.loc[(df.index > '04-01-2015') & (df.index < '04-08-2015')]['prediction'] \
    .plot(style='.')
plt.legend(['Truth Data','Prediction'])
plt.show()

In [ ]:
score = np.sqrt(mean_squared_error(test['_hum'], test['prediction']))
print(f'RMSE Score on Test set: {score:0.2f}')

In [ ]:
'''Calculate Error
Look at the worst and best predicted days'''
test['error'] = np.abs(test[TARGET] - test['prediction'])
test['date'] = test.index.date
test.groupby(['date'])['error'].mean().sort_values(ascending=False).head(10)